In [7]:
import pandas as pd
import plotly.express as px

metrics = pd.read_csv('metrics.csv', header=0, names=('start_time', 'inference_time'))

fig = px.histogram(metrics, x="inference_time", nbins=200)
fig.show()

In [8]:
fig2 = px.scatter(metrics, x="start_time", y="inference_time")
fig2.show()